In [2]:
!pip install gtts


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import tkinter as tk
from tkinter import filedialog, ttk
from tkinter import font as tkFont
from gtts import gTTS
import PyPDF2

# Function to create a custom message box
def custom_messagebox(title, message, bg_color):
    # Function to close the custom message box
    def on_ok():
        custom_box.destroy()

    # Create a new window for the custom message box
    custom_box = tk.Toplevel(root)
    custom_box.title(title)
    custom_box.configure(bg=bg_color)
    
    # Label to display the message
    label = tk.Label(custom_box, text=message, bg=bg_color, fg="white", font=button_font)
    label.pack(pady=20, padx=20)
    
    # OK button to close the message box
    ok_button = tk.Button(custom_box, text="OK", command=on_ok, bg="white", fg=bg_color, font=button_font)
    ok_button.pack(pady=10)

    # Make the message box modal (block interaction with the main window)
    custom_box.transient(root)
    custom_box.grab_set()
    root.wait_window(custom_box)

# Function to convert PDF to speech 
def convert_pdf_to_audio(pdf_file_path, language_code='en', speed='Medium'):
    try:
        # Opening the PDF file
        with open(pdf_file_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            # Getting the number of pages
            num_pages = len(pdf_reader.pages)
            
            # Extract text from each page
            text_list = []
            for i in range(num_pages):
                page = pdf_reader.pages[i]
                text = page.extract_text()
                if text:
                    text_list.append(text)
            
            # Convert text from all the pages into a single string
            text_string = " ".join(text_list)
            
            # Error message if no text found
            if not text_string.strip():
                custom_messagebox("No Text Found", "No text found in the PDF. Please select a PDF with readable text.", "red")
                return
            
            # Determine the speed based on the user selection
            if speed == 'Slow':
                speed_value = True  # Slow speed
            elif speed == 'Medium':
                speed_value = None  # Normal speed (default)
            elif speed == 'Fast':
                speed_value = False  # Fast speed
            # Adjust speed slightly faster
            if speed_value is not None:
                speed_value *= 0.85 
                
            # Generating audio from text using gTTS
            tts = gTTS(text=text_string, lang=language_code, slow=speed_value)
            audio_file = filedialog.asksaveasfilename(defaultextension=".mp3",
                                                      filetypes=[("MP3 files", "*.mp3")])
            
            # Save the MP3 file if a location was specified
            if audio_file:
                tts.save(audio_file)
                custom_messagebox("Conversion Success", f"Audio saved successfully as '{audio_file}'", "green")
            else:
                custom_messagebox("Conversion Canceled", "Audio file conversion was canceled.", "black")
    
    except Exception as e:
        custom_messagebox("Conversion Error", f"An error occurred during conversion: {str(e)}", "red")

# Function to handle file selection and display file path
def select_pdf_file():
    file_path = filedialog.askopenfilename(defaultextension=".pdf",
                                           filetypes=[("PDF files", "*.pdf")])
    if file_path:
        selected_file.set(file_path)
        convert_button.config(state=tk.NORMAL)
        file_label.config(text=f"Selected File: {file_path}")

# Function to initiate conversion after file selection
def start_conversion():
    pdf_file_path = selected_file.get()
    if not pdf_file_path:
        custom_messagebox("File Not Selected", "Please select a PDF file first.", "red")
        return
    convert_pdf_to_audio(pdf_file_path)

# Create the main tkinter window
root = tk.Tk()
root.title("PDF to Speech Converter")

# Custom font settings
title_font = tkFont.Font(family="Helvetica", size=18, weight="bold")
button_font = tkFont.Font(family="Helvetica", size=12)

# String variable to store selected file name
selected_file = tk.StringVar()

# Widgets with custom font
label = tk.Label(root, text="PDF to Speech Converter", font=title_font, fg="#756AB6")
label.pack(pady=20)

# Button to select file
select_button = tk.Button(root, text="Select PDF File", font=button_font, bg="#BEADFA", fg="black", padx=10, pady=5, command=select_pdf_file)
select_button.pack(pady=10)

# Label to display selected file path
file_label = tk.Label(root, text="Selected File: None", font=button_font, fg="#2E8B57")
file_label.pack(pady=5)

# Button to convert the PDF to audio file
convert_button = tk.Button(root, text="Convert to Audio", font=button_font, bg="#BEADFA", fg="black", padx=10, pady=5, state=tk.DISABLED, command=start_conversion)
convert_button.pack(pady=20)

# Run the tkinter main loop
root.mainloop()
